In [3]:

from variables import sourcepath
import os
import pandas as pd
import numpy as np
import etl.dim_author as auth
import etl.dim_journal as jour
import etl.common_functions as cof
import etl.dim_paper as pape
import etl.database as db
from db_credentials import dwh_db_connection_params
import roman
import datetime


In [4]:
eng, psycop2connect=db.initialize_engine(connection_params=dwh_db_connection_params)
paragraphs_in_dwh=db.load_full_table(eng, 'dim_paragraph')
